In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, Dropout
from tensorflow.keras import layers 
from tensorflow.keras import Model
import efficientnet.keras as efn
from tensorflow.keras import optimizers
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import seaborn as sns

from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from numpy import expand_dims

ModuleNotFoundError: No module named 'tensorflow'

## If you dont have the folder split do this step before creating paths to images

In [7]:
! pip install split-folders

In [ ]:
import splitfolders as sf
sf.ratio("Images", output="out", seed=1337, ratio=(.8, .1, .1), group_prefix=None)

In [2]:
types = ['Carriers','Cruise Ships','Destroyers','Fishing Boats','Kayaks','Sailboats','Security Boats','Submarines','Tugs']

# Work done in google co-lab with train/val/ and test folder in drive prior to running notbook
train_path = 'drive/MyDrive/Capstone 2/Images/train'
valid_path = 'drive/MyDrive/Capstone 2/Images/val'
test_path = 'drive/MyDrive/Capstone 2/Images/test'
bs = 6

train_batches = ImageDataGenerator(rescale=1./255.).flow_from_directory(train_path, target_size=(224, 224), classes=types, batch_size=bs)
valid_batches = ImageDataGenerator(rescale=1./255.).flow_from_directory(valid_path, target_size=(224, 224), classes=types, batch_size=bs)
test_batches = ImageDataGenerator(rescale=1./255.).flow_from_directory(test_path, target_size=(224, 224), classes=types, batch_size=bs)

NameError: name 'ImageDataGenerator' is not defined

In [3]:
# Model 1
vgg16_model = tf.keras.applications.vgg16.VGG16()

Dir_Class_Model = Sequential()

for layer in vgg16_model.layers[:-1]:
    Dir_Class_Model.add(layer)
    
for layer in Dir_Class_Model.layers[:-1]:
    layer.trainable = False
    
num_classes = 9
Dir_Class_Model.add(Dense(num_classes, activation='softmax'))

# Dir_Class_Model.summary()

opt = tf.keras.optimizers.SGD(learning_rate=0.001)

Dir_Class_Model.compile(
    optimizer=opt,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

Dir_Class_Model.fit(train_batches, validation_data=valid_batches, epochs=8, verbose=1)

NameError: name 'tf' is not defined

In [4]:
#Model 2

# weights can be changed to imagenet
base_model = efn.EfficientNetB0(input_shape = (224, 224, 3), include_top = False, weights = 'noisy-student')

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
predictions = Dense(9, activation="softmax")(x)
model_final = Model(base_model.input,predictions)

model_final.compile(optimizers.RMSprop(lr=0.0001,decay=1e-6),loss='categorical_crossentropy',metrics=['accuracy'])

eff_history = model_final.fit_generator(train_batches, validation_data=valid_batches, steps_per_epoch=120, epochs=10)

NameError: name 'efn' is not defined

In [6]:
# Model 3

last_model = efn.EfficientNetB7(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')
for layer in last_model.layers:
    layer.trainable = False

x1 = last_model.output
x1 = Flatten()(x1)
x1 = Dense(32, activation="selu")(x1)
x1 = Dropout(0.5)(x1)
predictions = Dense(9, activation="softmax")(x1)
model_final_2 = Model(last_model.input,predictions)
opt = optimizers.RMSprop(lr=0.0001,decay=1e-6)
model_final_2.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy'])

eff_history_2 = model_final_2.fit(train_batches, validation_data=valid_batches, steps_per_epoch=120, epochs=20)

NameError: name 'efn' is not defined

### Best model was EfficientNetB0. Getting example of first convolutional layer

In [ ]:
model = efn.EfficientNetB0(input_shape = (224, 224, 3), include_top = False, weights = 'noisy-student')
for i in range(len(model.layers)):
	layer = model.layers[i]
	# check for convolutional layer
	if 'conv' in layer.name:
		print(i, layer.name, layer.output.shape)
        
img = load_img('drive/MyDrive/Capstone 2/out/test/Sailboats/131.jpg', target_size=(224, 224))
pyplot.figure(figsize=(20,20))
# convert the image to an array
img = img_to_array(img)
# expand dimensions so that it represents a single 'sample'
img = expand_dims(img, axis=0)
# prepare the image (e.g. scale pixel values for the vgg)
img = preprocess_input(img)
# get feature map for first hidden layer
feature_maps = model.predict(img)
square = 5
for fmap in feature_maps:
	# plot all 64 maps in an 8x8 squares
	ix = 1
	for _ in range(square):
		for _ in range(square):
			# specify subplot and turn of axis
			ax = pyplot.subplot(square, square, ix)
			ax.set_xticks([])
			ax.set_yticks([])
			# plot filter channel in grayscale
			pyplot.imshow(fmap[:, :, ix-1])
			ix += 1
	# show the figure
pyplot.savefig('no_added_color_layer.jpg')
pyplot.show()

In [ ]:
image_plot_predict(rows=2, batch=test_batches,model=model_final)

In [5]:
actual, predictions, pics = con_fu(20,test_batches,model_final)
score = classification_report(actual, predictions,output_dict=True)
Precision_recall_f1_support = pd.DataFrame(score)


NameError: name 'con_fu' is not defined

In [8]:

# To get confusion matrix without Tensor Board

cf_matrix = confusion_matrix(predictions, actual, labels=[0,1,2,3,4,5,6,7,8])
accuracy = accuracy_score(predictions, actual)
# precision = precision_score(y_pred, y_test)
# reacall = recall_score(y_pred, y_test)
# f1 = f1_score(y_pred, y_test)
group_counts = ["{0:0.0f}\n".format(value) for value in cf_matrix.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in cf_matrix.flatten()/np.sum(cf_matrix)]
box_labels = [f"{v1}{v2}".strip() for v1, v2 in zip(group_counts,group_percentages)]
box_labels = np.asarray(box_labels).reshape(cf_matrix.shape[0],cf_matrix.shape[1])
plt.figure(figsize = (12, 8))
plt.rcParams['font.size'] = 14
sns.heatmap(cf_matrix, annot=box_labels, fmt='', cmap='Blues',xticklabels=types,yticklabels=types)
plt.yticks(va = 'center')
plt.xlabel('True Label', FontSize = 20)
plt.ylabel('Predicted Label', FontSize = 20)
plt.title('Accuracy: {:.2%}'.format(accuracy))
plt.tight_layout()
plt.savefig('BestModelCF.jpg',)

NameError: name 'confusion_matrix' is not defined

# Model with Fishing Boats and Secuirty Boats combined

In [11]:
types2 = ['Carriers','Cruise Ships','Destroyers','Kayaks','Sailboats','Submarines','Tugs','Utility Boat']

# Work done in google co-lab with train/val/ and test folder in drive prior to running notbook
train_path = 'drive/MyDrive/Capstone 2/out3/train'
valid_path = 'drive/MyDrive/Capstone 2/out3/val'
test_path = 'drive/MyDrive/Capstone 2/out3/test'
bs = 6

utility_train_batches = ImageDataGenerator(rescale=1./255.).flow_from_directory(train_path, target_size=(224, 224), classes=types2, batch_size=bs)
utility_valid_batches = ImageDataGenerator(rescale=1./255.).flow_from_directory(valid_path, target_size=(224, 224), classes=types2, batch_size=bs)
utility_test_batches = ImageDataGenerator(rescale=1./255.).flow_from_directory(test_path, target_size=(224, 224), classes=types2, batch_size=bs)

NameError: name 'ImageDataGenerator' is not defined

In [9]:
utility_model = efn.EfficientNetB0(input_shape = (224, 224, 3), include_top = False, weights = 'noisy-student')

for layer in utility_model.layers:
    layer.trainable = False

x = utility_model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
predictions = Dense(8, activation="softmax")(x)
utility_model_final = Model(utility_model.input,predictions)

utility_model_final.compile(optimizers.RMSprop(lr=0.0001,decay=1e-6),loss='categorical_crossentropy',metrics=['accuracy'])

eff_utility = utility_model_final.fit(utility_train_batches, validation_data=utility_valid_batches, epochs=20)

NameError: name 'efn' is not defined

In [10]:
actual_, predictions_, pics_ = con_fu(20,utility_test_batches,utility_model_final)
score_ = classification_report(actual_, predictions_,output_dict=True)
cf_matrix_2 = confusion_matrix(predictions_, actual_, labels=[0,1,2,3,4,5,6,7])
s = pd.DataFrame(score_)
accuracy_ = accuracy_score(predictions_, actual_)
# precision = precision_score(y_pred, y_test)
# reacall = recall_score(y_pred, y_test)
# f1 = f1_score(y_pred, y_test)
group_counts = ["{0:0.0f}\n".format(value) for value in cf_matrix_2.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in cf_matrix_2.flatten()/np.sum(cf_matrix_2)]
box_labels = [f"{v1}{v2}".strip() for v1, v2 in zip(group_counts,group_percentages)]
box_labels = np.asarray(box_labels).reshape(cf_matrix_2.shape[0],cf_matrix_2.shape[1])
plt.figure(figsize = (12, 8))
plt.rcParams['font.size'] = 14
sns.heatmap(cf_matrix_2, annot=box_labels, fmt='', cmap='ocean',xticklabels=types2,yticklabels=types2)
plt.yticks(va = 'center')
plt.xlabel('True Label', FontSize = 20)
plt.ylabel('Predicted Label', FontSize = 20)
plt.title('Accuracy: {:.2%}'.format(accuracy_))

#plt.savefig('UtilityBoatCM.jpg')

NameError: name 'con_fu' is not defined

# Model of just Fishing Boats and Secuirty Boats

In [12]:
types3 = ['Fishing Boats','Security Boats']

# Work done in google co-lab with train/val/ and test folder in drive prior to running notbook
train_path = 'drive/MyDrive/Capstone 2/out4/train'
valid_path = 'drive/MyDrive/Capstone 2/out4/val'
test_path = 'drive/MyDrive/Capstone 2/out4/test'
bs = 6

two_train_batches = ImageDataGenerator(rescale=1./255.).flow_from_directory(train_path, target_size=(224, 224), classes=types3, batch_size=bs)
two_valid_batches = ImageDataGenerator(rescale=1./255.).flow_from_directory(valid_path, target_size=(224, 224), classes=types3, batch_size=bs)
two_test_batches = ImageDataGenerator(rescale=1./255.).flow_from_directory(test_path, target_size=(224, 224), classes=types3, batch_size=bs)

NameError: name 'ImageDataGenerator' is not defined

In [13]:
just_two = efn.EfficientNetB0(input_shape = (224, 224, 3), include_top = False, weights = 'noisy-student')

for layer in just_two.layers:
    layer.trainable = False

x9 = just_two.output
x9 = Flatten()(x9)
x9 = Dense(1024, activation="relu")(x9)
x9 = Dropout(0.5)(x9)
predictions = Dense(2, activation="softmax")(x9)
just_two_final = Model(just_two.input,predictions)

just_two_final.compile(optimizers.RMSprop(lr=0.0001,decay=1e-6),loss='categorical_crossentropy',metrics=['accuracy'])

eff_just_two = just_two_final.fit_generator(two_train_batches, validation_data=two_valid_batches, epochs=10)

NameError: name 'efn' is not defined

In [14]:
actual_2, predictions_2, pics_2 = con_fu(20,two_test_batches,just_two_final)
score_2 = classification_report(actual_2, predictions_2,output_dict=True)
cf_matrix_2_boats = confusion_matrix(predictions_2, actual_2, labels=[0,1])
s = pd.DataFrame(score_2)
accuracy_2 = accuracy_score(predictions_2, actual_2)
# precision = precision_score(y_pred, y_test)
# reacall = recall_score(y_pred, y_test)
# f1 = f1_score(y_pred, y_test)
group_counts = ["{0:0.0f}\n".format(value) for value in cf_matrix_2_boats.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in cf_matrix_2_boats.flatten()/np.sum(cf_matrix_2_boats)]
box_labels = [f"{v1}{v2}".strip() for v1, v2 in zip(group_counts,group_percentages)]
box_labels = np.asarray(box_labels).reshape(cf_matrix_2_boats.shape[0],cf_matrix_2_boats.shape[1])
plt.figure(figsize = (12, 8))
plt.rcParams['font.size'] = 14
sns.heatmap(cf_matrix_2_boats, annot=box_labels, fmt='', cmap='icefire',xticklabels=types3,yticklabels=types3)
plt.yticks(va = 'center')
plt.xlabel('True Label', FontSize = 20)
plt.ylabel('Predicted Label', FontSize = 20)
plt.title('Accuracy: {:.2%}'.format(accuracy_2))

#plt.savefig('TwoBoats.jpg')

NameError: name 'con_fu' is not defined